# Using Twitter API with Tweepy

To interface with Twitter API, we can use third-party package such as Tweepy.
To use the package, we will need to register and get keys from twitter developer portal.  Then, we use these keys to authenticate with OAuth2 to access twitter API.

In [1]:
import tweepy
import pandas as pd
import pytz
import yaml

In [2]:
with open('twitter.yml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
auth = tweepy.OAuth2BearerHandler(config['bearer_token'])
api = tweepy.API(auth, wait_on_rate_limit=True)

Tweepy provides many features:
- searching and listing users' information
- reading tweets from user timelines
- creating, fetching, retweeing tweets
- managing followers
- adding and removing likes
- blocking users
- searching and filtering tweets
- listing trends
- streaming tweets in real-time

Let's create a helper function to print a tweet.  The function prints only one line containing, timestamp (in BKK timezone), user who creates tweet, and the content of the tweet.

In [4]:
def print_tweet(tweet):
    # tweet.created_at is an "unaware" timezone, but it acutally is a UTC timezone
    # we will have to make it a UTC first, then convert it to bkk
    utc_dt = tweet.created_at.replace(tzinfo=pytz.UTC)
    timezone_bkk = pytz.timezone('Asia/Bangkok')
    bkk_dt = utc_dt.astimezone(timezone_bkk)
    print('{} [{}] {}'.format(bkk_dt, tweet.user.name, tweet.text[:50]))

## Getting user's information
With get_user method, we can access user's public information such as screen_name, description, followers' count, etc.  Please refer to [Python – User object in Tweepy](https://www.geeksforgeeks.org/python-user-object-in-tweepy/) for more details of user object.

In [5]:
user = api.get_user(screen_name='katyperry')

In [6]:
print('Name:', user.name)
print('Screen Name:', user.screen_name)
print('Description:', user.description)
print('Number of Followers:', user.followers_count)
print('Number of Followings:', user.friends_count)
print('Number of Tweets/Retweets:', user.statuses_count)
print('Headshot\'s URL', user.profile_image_url)

Name: KATY PERRY
Screen Name: katyperry
Description: LOVE is the key that unlocks every door🗝️♥️
Number of Followers: 108556823
Number of Followings: 242
Number of Tweets/Retweets: 11823
Headshot's URL http://pbs.twimg.com/profile_images/1392465354622791687/w_KwtKcE_normal.jpg


## Accessing user timelines
We can get tweets, replies, and mentions in user's timeline, as long as it is public.

In [7]:
timeline = api.user_timeline(user_id=user.id)
for tweet in timeline:
    print_tweet(tweet)

2023-03-10 05:28:43+07:00 [KATY PERRY] 📣 T-minus 1 month until Easter! 📣 get ur 🐰 baskets
2023-03-08 00:42:53+07:00 [KATY PERRY] That feeling when the first styles from my Spring/
2023-03-06 11:02:04+07:00 [KATY PERRY] Fairy 🧚‍♂️ godmother or Perry godmother? Vote belo
2023-03-06 08:01:07+07:00 [KATY PERRY] anyway watch #idol rn and let me know what YOU did
2023-03-06 08:00:42+07:00 [KATY PERRY] #idol is celebrating its 21st bday this year. When
2023-03-06 05:01:43+07:00 [KATY PERRY] Good girl gone PLAID (or at least back to prismati
2023-03-01 10:21:44+07:00 [KATY PERRY] No such thing as too hip to be square when square 
2023-02-27 10:05:46+07:00 [KATY PERRY] 💔 #americanidol https://t.co/wy8IEDbo9G
2023-02-27 09:29:42+07:00 [KATY PERRY] You’re welcome Nicole Richie 🙃 #idol https://t.co/
2023-02-27 08:02:45+07:00 [KATY PERRY] .@lionelrichie:
@rockhall inductee ✔️
AMAs Icon Aw
2023-02-27 05:02:15+07:00 [KATY PERRY] No matter the weather, it’s always a good time for
2023-02-25 05:05:06+0

## Listing Trends
Trends are location-oriented.  We will have to use location coordinate to get trending information.

In [8]:
# Let's use BKK location
location_info = api.closest_trends(13.739060668870644, 100.53214799610562)

In [9]:
location_info

[{'name': 'Bangkok',
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/1225448',
  'parentid': 23424960,
  'country': 'Thailand',
  'woeid': 1225448,
  'countryCode': 'TH'}]

Get trending keywords and put them in dataframe

In [10]:
trendings = api.get_place_trends(location_info[0]['woeid'])
df = pd.DataFrame(trendings[0]['trends'])

In [11]:
df

,name,url,promoted_content,query,tweet_volume
0,#จุงอาเชน,http://twitter.com/search?q=%23%E0%B8%88%E0%B8...,None,%23%E0%B8%88%E0%B8%B8%E0%B8%87%E0%B8%AD%E0%B8%...,30668.0
1,#dunknatachai,http://twitter.com/search?q=%23dunknatachai,None,%23dunknatachai,35412.0
2,#จุงดัง,http://twitter.com/search?q=%23%E0%B8%88%E0%B8...,None,%23%E0%B8%88%E0%B8%B8%E0%B8%87%E0%B8%94%E0%B8%...,18349.0
3,#ErrrDay_With_Youngjae,http://twitter.com/search?q=%23ErrrDay_With_Yo...,None,%23ErrrDay_With_Youngjae,43661.0
4,#THEDREAMSHOW2_in_BKK,http://twitter.com/search?q=%23THEDREAMSHOW2_i...,None,%23THEDREAMSHOW2_in_BKK,1664390.0
5,TENsSCENT OF FREEDOM,http://twitter.com/search?q=%22TENsSCENT+OF+FR...,None,%22TENsSCENT+OF+FREEDOM%22,626914.0
6,BamBam All the Butlers EP249,http://twitter.com/search?q=%22BamBam+All+the+...,None,%22BamBam+All+the+Butlers+EP249%22,43172.0
7,JAM X FILM,http://twitter.com/search?q=%22JAM+X+FILM%22,None,%22JAM+X+FILM%22,52321.0
8,patrick thaifest 2023,http://twitter.com/search?q=%22patrick+thaifes...,None,%22patrick+thaifest+2023%22,10549.0
9,Nene Diary De La Lune,http://twitter.com/search?q=%22Nene+Diary+De+L...,None,%22Nene+Diary+De+La+Lune%22,191886.0


In [12]:
# sort trending based on their tweet volume in descending order
trending_df = df.sort_values('tweet_volume', ascending=False)
trending_df

,name,url,promoted_content,query,tweet_volume
4,#THEDREAMSHOW2_in_BKK,http://twitter.com/search?q=%23THEDREAMSHOW2_i...,None,%23THEDREAMSHOW2_in_BKK,1664390.0
30,#CUTIEPIECONCERT,http://twitter.com/search?q=%23CUTIEPIECONCERT,None,%23CUTIEPIECONCERT,1041692.0
35,#YSLBeautyxTEN,http://twitter.com/search?q=%23YSLBeautyxTEN,None,%23YSLBeautyxTEN,718955.0
5,TENsSCENT OF FREEDOM,http://twitter.com/search?q=%22TENsSCENT+OF+FR...,None,%22TENsSCENT+OF+FREEDOM%22,626914.0
22,Silicon Valley Bank,http://twitter.com/search?q=%22Silicon+Valley+...,None,%22Silicon+Valley+Bank%22,377751.0
15,LALISA GUNCANG JAKARTA,http://twitter.com/search?q=%22LALISA+GUNCANG+...,None,%22LALISA+GUNCANG+JAKARTA%22,368569.0
16,USDC,http://twitter.com/search?q=USDC,None,USDC,324559.0
36,#OnceLAZ1_GoodbyeParty,http://twitter.com/search?q=%23OnceLAZ1_Goodby...,None,%23OnceLAZ1_GoodbyeParty,305078.0
25,WONWOO,http://twitter.com/search?q=WONWOO,None,WONWOO,216228.0
9,Nene Diary De La Lune,http://twitter.com/search?q=%22Nene+Diary+De+L...,None,%22Nene+Diary+De+La+Lune%22,191886.0


In [13]:
# get the keyword with the highest tweet volume
top_trending_keyword = trending_df.iloc[3]['name']
print('Top Trending Keyword = ', top_trending_keyword)

Top Trending Keyword =  TENsSCENT OF FREEDOM


In [14]:
tweets = api.search_tweets(q=top_trending_keyword, count=10)

In [15]:
for tweet in tweets:
    print_tweet(tweet)

2023-03-12 17:36:57+07:00 [💜TenLee.960207🐱] RT @TENzenter: 230312 tatlermalaysia Instagram Upd
2023-03-12 17:36:57+07:00 [˳พานิพานิล์✧༚] RT @prettyjuuu: แจก 100฿ 

มาเอา 1M ให้น้องเตนล์กั
2023-03-12 17:36:56+07:00 [tengerine] RT @NamwanPlastx: อันนี้ที่แฟนขอชนกำปั้น แต่น้อมเต
2023-03-12 17:36:56+07:00 [G~ 🦁🐱] RT @ywnpsk: น้องเตนล์ถึงเกาหลีแล้ว คนเก่ง รับจดหมา
2023-03-12 17:36:53+07:00 [mypreciousTEN ft so10mini 🔥] @kihae_129 น้องเตนล์เก่ง เลิฟลี่เก่ง 

TENsSCENT O
2023-03-12 17:36:53+07:00 [˳พานิพานิล์✧༚] RT @1OO1times: พี่เตนเค้าปังมากงานเปิดชอป ysl beau
2023-03-12 17:36:52+07:00 [moon] RT @Fz_Fang: มา ขอคนละรีพลายค่ะ

TENsSCENT OF FREE
2023-03-12 17:36:51+07:00 [ไข่ต้มโยโย่💋🥚] RT @itsdefp: แจก 100 บาทรีทวิต+เมนชั่นติดแท็ก (ถ้า
2023-03-12 17:36:51+07:00 [Aisᵛ | k.ia] RT @TEN10LAY: Baby, your sweet smile, your 10vely 
2023-03-12 17:36:50+07:00 [M̶y̶ ꪔ̤̮ 🕸️´ˎ˗.𓂃♡̴] RT @babyjey10: ก็เขียนไปเลยสิคะว่า Saint Laurent เ


## Understanding cursor
For Twitter API, those information will be returned with some certain limitations e.g. providing only 20 followers.  If you want more complete list, we will have to use Cursor API.  Cursor API is basically a paging mechanism.

In [16]:
screen_name = 'natawutn'
user = api.get_user(screen_name=screen_name)

In [17]:
print('Name:', user.name)
print('Screen Name:', user.screen_name)
print('Description:', user.description)
print('Number of Followers:', user.followers_count)
print('Number of Followings:', user.friends_count)
print('Number of Tweets/Retweets:', user.statuses_count)
print('Headshot\'s URL', user.profile_image_url)

Name: Natawut Nupairoj
Screen Name: natawutn
Description: Just a normal University lecturer who obsesses with gadgets and Liverpool FC
Number of Followers: 158
Number of Followings: 117
Number of Tweets/Retweets: 824
Headshot's URL http://pbs.twimg.com/profile_images/843009002388697093/4DztTUyP_normal.jpg


In [18]:
for friend in user.followers():
    print(friend.screen_name)

wanwan57932
mandzbgarcia
imrpremsingh
Olivia_36761
dlfkgzxlfl
putchonguth
supott
baktar0
noppadol_s
AngelaDunwoody
BatikThailand
DarkDrag0nite
uVno34bor7Y23Jy
felipehoffa
Remi_niscene
pacoid
aek13140588
raw_divyanshu
jonaml
poymee


In [19]:
# Iterate through all of the account friends
for friend in tweepy.Cursor(api.get_followers, screen_name=screen_name).items():
    # Process the friend here
    print(friend.screen_name)

wanwan57932
mandzbgarcia
imrpremsingh
Olivia_36761
dlfkgzxlfl
putchonguth
supott
baktar0
noppadol_s
AngelaDunwoody
BatikThailand
DarkDrag0nite
uVno34bor7Y23Jy
felipehoffa
Remi_niscene
pacoid
aek13140588
raw_divyanshu
jonaml
poymee
poymee
Loogkaewww
loogkaeww
VTTLive
rajesh_dangi
iattaphon
daiya_dian1m
amnuay
kRWcuGfiPAyLrE6
urai8617
fony_lew
ShivrajHulli
ToyJames
kaneungjit
pukaslim
Philsonia
CVsquad
Iamz
KiartP
athakorn
athakorn
indyprogrammer
LOO23NG
Atsadawutt
ThaiLFC
push7joshi
pasettha
LiverpoolFC
pornsuree
ErynLWhitteker
DungjitS
adatt56
ra_Chot
TM_NT
SalmonCroquette
kaminph
ohmohm
boom_hathai
PopPio88
NahCM0l4
vvace
Palarp72
Pxxxz
bentocast
nice54
iwhale
pidd01
KeereeW
NoOnGKead
aunonline
AnchuleeMoolthi
johnnydcx
niddy0509
niddy0905
scb_thailand
rukyimcom
teeraphanr
pataoengineer
vorapoap
ttumm
Boom_LFC
a_anakin
Princesss_Diary
Pakorn035
IAmOhs
Ming2551
belteus
eakkim
simply_rhythm
chittinat
P_tuktuk24
imkappu
luvnnsp
iahcarow
bungarm
popjedi
tiangnga
thaneth73
wpoonsiri
jeremy